In [1]:
from math import floor

import folium
import pandas as pd
from bson import ObjectId
import numpy as np
from pymongo import MongoClient
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
from pyproj import Transformer
from shapely.geometry.linestring import LineString
from shapely.geometry.multilinestring import MultiLineString
from shapely.geometry.point import Point
from folium import plugins
from folium import LinearColormap
from matplotlib import colors as mcolors
from folium.plugins import HeatMap

In [2]:
graph = ox.io.load_graphml("C:/Users/keyas/OneDrive/Documents/sih/map_matching_research/data/merged_graph.graphml")
# graph = ox.convert.to_undirected(graph)

In [3]:
nodes, edges = ox.graph_to_gdfs(graph)
print(nodes['avg_visibility_January'].iloc[0])


7.018716577540105


In [4]:
for month in ['avg_rainfall_January', 'avg_rainfall_February', 'avg_rainfall_March',
              'avg_rainfall_April', 'avg_rainfall_May', 'avg_rainfall_June',
              'avg_rainfall_July', 'avg_rainfall_August', 'avg_rainfall_September',
              'avg_rainfall_October', 'avg_rainfall_November', 'avg_rainfall_December']:
    nodes[month] = pd.to_numeric(nodes[month], errors='coerce')
season_mapping = {
    'winter': ['November', 'December', 'January', 'February'],
    'spring': ['March', 'April', 'May'],
    'summer': ['June', 'July', 'August'],
    'autumn': ['September', 'October', 'November']
}

In [5]:
def calculate_seasonal_rainfall(node_df, season_mapping):
    for season, months in season_mapping.items():
        rainfall_columns = [f'avg_rainfall_{month}' for month in months]
        node_df[f'avg_rainfall_{season}'] = node_df[rainfall_columns].mean(axis=1)
    return node_df

In [6]:
nodes = calculate_seasonal_rainfall(nodes, season_mapping)


In [7]:
def rainfall_to_color(rainfall, min_rainfall, max_rainfall):
    if pd.isna(rainfall):  
        return '#FFFFFF'  # white for missing data
    normalized_rainfall = (rainfall - min_rainfall) / (max_rainfall - min_rainfall)
    colormap = folium.LinearColormap(['#ff0000', '#ffff00'], vmin=0, vmax=1)
    return colormap(normalized_rainfall)


In [8]:
for season in season_mapping.keys():
    rainfall_column = f'avg_rainfall_{season}'
    min_rainfall = nodes[rainfall_column].min()
    max_rainfall = nodes[rainfall_column].max()
    
    print(f"Generating heatmap for {season} (min: {min_rainfall}, max: {max_rainfall})")

    m = folium.Map(location=[nodes['y'].mean(), nodes['x'].mean()], zoom_start=12)
    
    for node in nodes.index:
        rainfall = nodes.loc[node, rainfall_column]
        if pd.isna(rainfall):
            continue
        
        lat, lon = nodes.loc[node, ['y', 'x']]
        color = rainfall_to_color(rainfall, min_rainfall, max_rainfall)
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"Node: {node}, Rainfall: {rainfall:.2f} mm"
        ).add_to(m)
    
    file_name = f"{season}_rainfall_heatmap.html"
    m.save(file_name)
    print(f"Heatmap for {season} saved as {file_name}")
    m

Generating heatmap for winter (min: 0.030137130801687673, max: 4.859999999999998)
Heatmap for winter saved as winter_rainfall_heatmap.html
Generating heatmap for spring (min: 0.016560846560846467, max: 11.154501915708813)
Heatmap for spring saved as spring_rainfall_heatmap.html
Generating heatmap for summer (min: 0.17666666666666656, max: 26.775555555555552)
Heatmap for summer saved as summer_rainfall_heatmap.html
Generating heatmap for autumn (min: 0.08222222222222213, max: 8.607777777777777)
Heatmap for autumn saved as autumn_rainfall_heatmap.html


In [9]:
# for low rainfall = red, high rainfall = yellow